In [48]:
import numpy as np

from pandas import read_csv
from numba import njit
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from libs.print_bench import print_bench

In [49]:
dir_path = 'data/waypoints.csv'
df = read_csv(dir_path)
wx = df['y'].values
wy = df['x'].values

In [50]:
%%capture

calculate_closest_id = lambda x, y, point: np.argmin(np.hypot(point[0] - x, point[1] - y))
calculate_closest_id_jit = njit(calculate_closest_id)

input_transposed = np.array([wx, wy]).T
target = np.array([wx[-1], wy[-1]])
cdist_target = np.array([[wx[-1], wy[-1]]])

t_numpy   = %timeit -o calculate_closest_id(wx, wy, target)
t_numba   = %timeit -o calculate_closest_id_jit(wx, wy, target)
t_cdist   = %timeit -o cdist(input_transposed, cdist_target).argmin()
t_ckdtree = %timeit -o cKDTree(input_transposed).query(target, k=1, workers=-1)[1]

In [52]:
print_bench(['numpy', 'numba', 'cdist', 'ckdtree'], [t_numpy, t_numba, t_cdist, t_ckdtree])


Numpy test:
5.17 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numba test:
1.43 µs ± 32.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Cdist test:
5.8 µs ± 90 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Ckdtree test:
50.9 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Conclusion

Numba is really fast, but for 2D arrays, NumPy would be a lot easier to implement. cKDTree may be faster for higher dimensional arrays, but I haven't tested it.